# SLIC Formulation

The SLIC (Solvation Layer Interface Condition) formulation includes a nonlinear boundary condition for the electric potential at the solute surface and addresses the charge-hydration assymetry in the Poisson-Boltzmann Model. For more info please refer to: https://pubs.acs.org/doi/10.1021/acs.jctc.6b00832. This formulation, as the direct stern formulation, uses two surfaces for computing potentials and solvation energies. Here, we will use it to study the interaction energy between two ions of radius 1.27 and charge of +1 and -1. The ions have a 2Å Stern layer and are located 7.5Å appart.

In [ ]:
import pbj

In [ ]:
simulation = pbj.electrostatics.Simulation(formulation="slic", stern_layer = True)

`stern_layer=True`is redundant as the `slic`formulation will default to use a Stern layer. 

In [ ]:
solute_1 = pbj.electrostatics.Solute("pqrs/ion_1.pqr", mesh_generator="msms", mesh_density = 15.0)
solute_2 = pbj.electrostatics.Solute("pqrs/ion_2.pqr", mesh_generator="msms", mesh_density = 15.0)

In this simulation, we will use parameters that are slightly different than the defaults. 

In [ ]:
solute_1.ep_in = 1
solute_2.ep_in = 1
solute_1.stern_mesh_density_ratio = 0.2
solute_2.stern_mesh_density_ratio = 0.2
solute_1.slic_alpha = 0.94
solute_1.slic_beta = -18.72
solute_1.slic_gamma = -0.47
solute_2.slic_alpha = 0.94
solute_2.slic_beta = -18.72
solute_2.slic_gamma = -0.47

In [ ]:
simulation.gmres_tolerance = 1e-5
simulation.slic_max_iterations = 20
simulation.slic_tolerance = 1e-3
simulation.kappa = 1e-15
simulation.add_solute(solute_1)
simulation.add_solute(solute_2)

and now, we calculate

In [ ]:
pbj.electrostatics.simulation.bempp.api.enable_console_logging("info")
simulation.calculate_solvation_energy()

Checking out results

In [ ]:
total_solvation_energy = simulation.solutes[0].results["solvation_energy"] + simulation.solutes[1].results["solvation_energy"]
print(total_solvation_energy)

Next, we compute the solvation energy of each ion individually. The cool thing about SLIC is that it considers charge asymmetry, making the solvation energy of the +1 ion different from the -1 ion. This is not possible with standard PB.

In [ ]:
simulation_1 = pbj.electrostatics.Simulation(formulation="slic", stern_layer = True)
solute_1_alone = pbj.electrostatics.Solute("pqrs/ion_1.pqr", mesh_generator="msms", mesh_density = 15.0)
pbj.electrostatics.simulation.bempp.api.enable_console_logging("info")
solute_1_alone.ep_in = 1
solute_1_alone.stern_mesh_density_ratio = 0.2
solute_1_alone.slic_alpha = 0.94
solute_1_alone.slic_beta = -18.72
solute_1_alone.slic_gamma = -0.47
simulation_1.gmres_tolerance = 1e-5
simulation_1.slic_max_iterations = 20
simulation_1.slic_tolerance = 1e-3
simulation_1.kappa = 1e-15
simulation_1.add_solute(solute_1_alone)
simulation_1.calculate_solvation_energy()
energy_ion_p1 = simulation_1.solutes[0].results["solvation_energy"]

In [ ]:
print(energy_ion_p1)

In [ ]:
simulation_2 = pbj.electrostatics.Simulation(formulation="slic", stern_layer = True)
solute_2_alone = pbj.electrostatics.Solute("pqrs/ion_2.pqr", mesh_generator="msms", mesh_density = 15.0)
solute_2_alone.ep_in = 1
solute_2_alone.stern_mesh_density_ratio = 0.2
solute_2_alone.slic_alpha = 0.94
solute_2_alone.slic_beta = -18.72
solute_2_alone.slic_gamma = -0.47
simulation_2.gmres_tolerance = 1e-5
simulation_2.slic_max_iterations = 20
simulation_2.slic_tolerance = 1e-3
simulation_2.kappa = 1e-15
simulation_2.add_solute(solute_2_alone)
simulation_2.calculate_solvation_energy()
simulation_1.calculate_solvation_energy()
energy_ion_m1 = simulation_2.solutes[0].results["solvation_energy"]

In [ ]:
inter_energy = total_solvation_energy - energy_ion_p1 - energy_ion_m1
print(inter_energy)

## SLIC Prop. Formulation (work in progress)

The SLIC Prop. formulation it's a modification to the SLIC formulation that includes a new non-linear boundary condition in the outer surface. In most cases, the results are close to the SLIC formulation but, in cases where the total charge of the solute is zero, the SLIC Prop. formulation effectively computes the electric potential on the outer surface whereas the SLIC formulation isn't. For this example let's simulate a two atoms system with opposite charges:

In [ ]:
simulation = pbj.electrostatics.Simulation(formulation = 'slic_prop')
atoms = pbj.electrostatics.Solute('pqrs/2_atoms.pqr', mesh_density = 5.0 , mesh_generator = 'msms')
simulation.add_solute(atoms)

In [ ]:
simulation.solutes[0].ep_in = 1
simulation.kappa = 0
simulation.solutes[0].ep_stern = 80 
simulation.solutes[0].pb_formulation_stern_width = 2.0 
simulation.solutes[0].stern_mesh_density = 2

In [ ]:
simulation.slic_alpha = 0.94
simulation.slic_beta = -18.72
simulation.slic_gamma = -0.47

In [ ]:
simulation.calculate_potential(rerun_all = True)

In [ ]:
simulation.solutes[0].results['phi_stern'].plot()

We can see a clear dipole in the plot, just as it should be. If we compute the same value using the SLIC formulation:

In [ ]:
simulation.pb_formulation = 'slic'
simulation.calculate_potential(rerun_all = True)
simulation.solutes[0].results['phi_stern'].plot()

The dipole is no longer noticeable. 